In [1]:
# %% Import Modules
######## Import Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick  # Importing matplotlib ticker module for formatting
import sys
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from itertools import groupby
from lifelines import KaplanMeierFitter
from statsmodels.stats.proportion import proportion_confint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
import plotly
pio.templates.default = "plotly_white"
clrs = plotly.colors.DEFAULT_PLOTLY_COLORS
from qsdc.client import Client
import met_client as app
import genealogy
import genealogy_v2
import unit_cell_electrical_yield_and_metrics_with_rel as uceym_rel
import unit_cell_metro_metrics
import cell_tiering_metro
import mass
from qsdc.client import Client
# create the quantumscape data client
qs_client = Client()
# Query data
conn = qs_client.get_mysql_engine()

batches = "UCD003A[A-E]|UCD004A[F-G]|UCD005A[A-C]|UCD006A[A-B]|UCD006A[D-E]|UCD006A[G-H]|UCD006A[J-N]|UCD008AA|UCD011A[A-E]|UCD005AF|UCD006A[R-S]|UCD013AA" #UCD mastertracker
batches = "UCB003A[K-N]"#|UCB003AE-US00-24|UCB003AF-US00-24|UCB003AF-US00-26|UCB003AG-US00-02|UCB003AG-US00-15|UCB003AG-US00-34|UCB003AG-US00-35|"
batches = "UCB004A"
#batches = "UCB003AN"

c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\Archive\genealogy_v2.py:533: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_df[column_name] = None
c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\Archive\genealogy_v2.py:533: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_df[column_name] = None
c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\Archive\genealogy_v2.py:533: PerformanceWarnin

In [2]:
# %% Query Data
######## Query Data
# =============================================================================
# ========================         QUERY DATA          =======================
# =============================================================================

######## CELL GENEALOGY ########
# produce dataframe that lists all cells in the batches specified above 
df_sample_ids = genealogy.get_unit_cells(batches) # query ID's of unit cells that are in the batches mentioned above

# creates dataframe with the genealogy history of each unit cell
df_genealogy = genealogy.get_genealogy_unitcell(df_sample_ids["US_id"]) # queries the genealogy history of every sell in the batch
df_samples = df_sample_ids.merge(df_genealogy, on="US_id", how="left") # merges the list of unit cells with the geneology dataframe 


######## QUERY ELECTRICAL YIELD AND ELECTRICAL METRICS ########
# get yield and electrical metrics for each unit cells
df_electrical_yield_metrics = uceym_rel.get_electrical_yield_and_metrics(df_samples["US_id"])
df_testdata = df_samples.merge(df_electrical_yield_metrics, on="US_id") #merges this list with the previous data frame
df_testdata = df_testdata.sort_values('US_id')


######## METROLOGY DATA 
agent = app.ImageAgent()
# get anode metrics from hifi scds scans
anode_metrics = unit_cell_metro_metrics.get_anode_tier(
    df_testdata["US_id"].str.slice(stop=13).unique(), agent
)
anode_metrics.rename(columns={"sample": "US_id"}, inplace=True)
# get radiograph metrics from nordson matrix x-ray scans
radiograph_metrics = unit_cell_metro_metrics.get_radiograph_tier(
    df_testdata["US_id"].str.slice(stop=13).unique(), agent
)
radiograph_metrics.rename(columns={"sample": "US_id"}, inplace=True)
# get unit cell mass metrics
mass_metrics = mass.get_mass_data(df_testdata["US_id"])
# unit cell tiering (alpha 2 criteria)
df_testdata = cell_tiering_metro.cell_tiering(
    df_testdata, anode_metrics, radiograph_metrics, mass_metrics
)

100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


In [3]:
# %% Plot Screen UCT Yields
######## Plot Screen UCT Yields
#  =============================================================================
# ========================      SCREEN UCT YIELDS        =======================
# =============================================================================

##Take dataframe for individual cell and group them by batch ("process")
grouping = "process"
data = df_testdata.copy() #make copy


#data = data[~data['US_id'].str.contains("APD224A|APD238AE")] #focus on subset by filtering out

#create new dataframe where cells from the same batches are grouped together
df_cyield = (
    data[[grouping]]
    .groupby(grouping)
    .first()
    .join(
        data[
            [
                grouping,
                "Build Count",
                "Formation Count",
                "C/3 Count",
                "1C Count",
                "Fast Charge Count",
                "4C Count",
                "Final 1C Count",
                "Yield Count",
                # "Anode Count",
                # "Anode Yield Count",
                # "Anode Yield Tier 1 Count",
                # "Ultrasound Count",
                # "Ultrasound Yield Count",
                # "Metrology Count",
                # "Tier 1 Count",
                # "Tier 1+2 Count",
            ]
        ]
        .groupby(grouping)
        .sum(),
        how="right",
    )
    .reset_index()
).set_index(grouping)


df_cyield[
    [
        "Cells Built",
        "Formation Yield",
        "C/3 Yield",
        "1C Yield",
        "Fast Charge Yield",
        "4C Yield",
        "Final 1C Yield",
        "Screen Yield",
    ]
] = 100 * df_cyield[
    [
        "Build Count",
        "Formation Count",
        "C/3 Count",
        "1C Count",
        "Fast Charge Count",
        "4C Count",
        "Final 1C Count",
        "Yield Count",
    ]
].div(
    df_cyield["Build Count"], axis=0
)

# df_cyield["Anode Yield Tier 1+2"] = (
#     df_cyield["Anode Yield Count"] / df_cyield["Anode Count"]
# ) * df_cyield["Screen Yield"]

# df_cyield["Anode Yield Tier 1"] = (
#     df_cyield["Anode Yield Tier 1 Count"] / df_cyield["Anode Count"]
# ) * df_cyield["Screen Yield"]

# df_cyield["Tier 1+2 Yield"] = (
#     df_cyield["Tier 1+2 Count"] / df_cyield["Metrology Count"]
# ) * df_cyield["Screen Yield"]

# df_cyield["Tier 1 Yield"] = (
#     df_cyield["Tier 1 Count"] / df_cyield["Metrology Count"]
# ) * df_cyield["Screen Yield"]


df_cyield = df_cyield.sort_values(grouping)

fig = go.Figure(
    data=[
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["Cells Built"],
            name="Cells Built",
        ),
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["Formation Yield"],
            name="Formation Yield",
        ),
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["1C Yield"],
            name="1C Yield",
        ),
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["Fast Charge Yield"],
            name="2.5C Yield",
        ),
        # go.Bar(
        #     x=df_cyield.index,
        #     y=df_cyield["4C Yield"],
        #     name="4C Yield",
        # ),
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["Final 1C Yield"],
            name="Final 1C Yield",
        ),
        go.Bar(
            x=df_cyield.index,
            y=df_cyield["C/3 Yield"],
            name="C/3 Yield",
        ),
        # go.Bar(
        #     x=df_cyield.index,
        #     y=df_cyield["Anode Yield Tier 1+2"],
        #     name="Anode 1+2 Yield",  # (HiFi, Ultran, Mass, Radiograph)",
        # ),
    #     go.Bar(
    #         x=df_cyield.index,
    #         y=df_cyield["Anode Yield Tier 1"],
    #         name="Anode Yield (Tier 1)",  # (HiFi, Ultran, Mass, Radiograph)",
    #     ),
    ],
)

# bar group mode
fig.update_layout(barmode="group")

fig.data[0].text = [f"N= {n}" for n in df_cyield["Build Count"]]
fig.update_traces(textposition="inside", textfont_size=15)

fig.update_layout(
    xaxis_title=grouping,
    yaxis_title="Screen yield (%)",
    # y axis font size = 14, x axis font size = 11
    font=dict(size=14),
    legend={"title_text": ""},
    yaxis_range=[0, 101],
)


# fig.update_xaxes(
#     categoryorder="array",
#     categoryarray=data.sort_values(["cell_build_date", "batch"])[grouping].unique(),
# )

fig.update_xaxes(
    categoryorder="array",
    categoryarray=data.sort_values(["batch"])[grouping].unique(),
)


# change the bar colors
colors = [
    px.colors.qualitative.Plotly[2],
    px.colors.qualitative.Plotly[5],
    px.colors.qualitative.Plotly[6],
    px.colors.qualitative.Plotly[4],
    # px.colors.qualitative.Plotly[7],
    px.colors.qualitative.Plotly[0],
    px.colors.qualitative.Plotly[3],
    px.colors.qualitative.Plotly[1],
]
for i in range(len(fig.data)):
    fig.data[i].marker.color = colors[i]

fig.show(renderer="browser")

In [4]:
# %% Plot Cell Metrics
######## Plot Cell Metrics
# =============================================================================
# ========================        CELL METRICS          =======================
# =============================================================================

grouping = "process"
color_by = "experiment"

data = df_testdata.copy()


fig = make_subplots(
    1,
    2,
    horizontal_spacing=0.12,
    vertical_spacing=0.1,
    shared_xaxes=True,
)

# create a color dictionary for each color_by category
color = dict(zip(data[color_by].unique(), px.colors.qualitative.Plotly))

# Set a flag to ensure legend items are added only once
legend_added = {key: False for key in data[color_by].unique()}

for label, group in data.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        fig.add_trace(
            go.Box(
                x=group_color[grouping],
                y=group_color["AMSDischargeCapactiy_Co3"],
                quartilemethod="linear",
                name=color_value,
                text=group_color["US_id"],
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            1,
            1,
        )
        legend_added[color_value] = True


fig.update_yaxes(
    title_text="1C Discharge Capacity (mAh/g)",
    range=[190, 205],
    row=1,
    col=1,
)

for label, group in data.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        fig.add_trace(
            go.Box(
                x=group_color[grouping],
                y=group_color[
                    "ADDischargeCapactiy_Co3"
                ],  # [group["Final 1C Count"] == 1]
                quartilemethod="linear",
                name=color_value,
                text=group_color["US_id"],
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            1,
            2,
        )
        legend_added[color_value] = True


fig.update_yaxes(
    title_text="C/3 Discharge Capacity (mAh/cm<sup>2</sup>)",
    range=[5, 7],
    row=1,
    col=2,
)

for i in range(2):
    fig.update_yaxes(
        showline=True,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        row=1,
        col=i + 1,
    )
    fig.update_xaxes(
        showline=True,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        row=1,
        col=i + 1,
    )

fig.update_layout(
    title_text="",
    # xaxis_title=grouping,
    font=dict(
        size=16,
    ),
)

fig.update_traces(boxpoints="all", jitter=0.1)

fig.update_xaxes(
    categoryorder="array",
    categoryarray=data.sort_values(["cell_build_date", "batch"])[grouping].unique(),
)

fig.show(renderer="browser")


fig = make_subplots(
    1,
    2,
    horizontal_spacing=0.12,
    vertical_spacing=0.1,
    shared_xaxes=True,
)

# plot colors in px.colors.qualitative.Plotly
color = dict(zip(data[color_by].unique(), px.colors.qualitative.Plotly))


# Set a flag to ensure legend items are added only once
legend_added = {key: False for key in data[color_by].unique()}

for label, group in data.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        fig.add_trace(
            go.Box(
                x=group_color[grouping],
                y=group_color[
                    "MedDischargeASR_form"
                ],  # [group["Formation Count"] == 1]
                quartilemethod="linear",
                name=color_value,
                text=group_color["US_id"],
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            1,
            1,
        )
        legend_added[color_value] = True


fig.update_yaxes(
    title_text="Formation Discharge ASR (Ohm cm<sup>2</sup>)",
    range=[20, 30],
    row=1,
    col=1,
)

for label, group in data.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        fig.add_trace(
            go.Box(
                x=group_color[grouping],
                y=group_color["MedDischargeASR_1C"],  # [group["Final 1C Count"] == 1]
                quartilemethod="linear",
                name=color_value,
                text=group_color["US_id"],
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            1,
            2,
        )
        legend_added[color_value] = True


fig.update_yaxes(
    title_text="1C Discharge ASR (Ohm cm<sup>2</sup>)",
    range=[20, 30],
    row=1,
    col=2,
)

for i in range(2):
    fig.update_yaxes(
        showline=True,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        row=1,
        col=i + 1,
    )
    fig.update_xaxes(
        showline=True,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        row=1,
        col=i + 1,
    )


fig.update_layout(
    # xaxis_title=grouping,
    font=dict(
        size=16,
    ),
    # show legend
    showlegend=True,
)

fig.update_traces(boxpoints="all", jitter=0.1)

fig.update_xaxes(
    categoryorder="array",
    categoryarray=data.sort_values(["batch"])[grouping].unique(),
)  #

fig.show(renderer="browser")

In [5]:
# %% Create dataframe that will feed into the Cell Tiering script
######## Tabulate Yielded Cells
#Filter yielded Cells
data = df_testdata[df_testdata['Yield Count'] == 1]
data = data.rename(columns={"US_id": "Cell ID"})
data = data.rename(columns={"process": "Batch"})

#Generate new dataframe to input into Cell Tiering Function
YieldedCells = data[data['Yield Count'] == 1]
YieldedCells = YieldedCells[["Cell ID","Batch"]]

# List of columns to add
new_columns = [
    "Cell Status", "Pairing Group", "Edge Wetting", "Thickness", "Alignment", 
    "Anode", "DischargeASR_1C", "Capacity_Co3", 
    "dVdt_fastcharge", "total_rank_score", "Tray", "TrayRow", "TrayColumn"
]
# Add each column with NaN values
for col in new_columns:
    YieldedCells[col] = np.nan

#Update if yielded cell is on 2L Reliability
data = data[["Cell ID","Reliability Test Count"]]
# Merge the two dataframes on the 'Cell ID' column
merged_df = pd.merge(YieldedCells, data, on='Cell ID', how='right')
# Update 'Cell Status' where 'Reliability Test Count' is 1 in merged dataframe
merged_df.loc[merged_df['Reliability Test Count'] == 1, 'Cell Status'] = '2L Reliability'
# Drop the added columns to keep only the original columns from YieldedCells
YieldedCells = merged_df.drop(columns=['Reliability Test Count'])

#Update if yielded cell is on already in a multilayer pouch
# Query dataframe from database
gen = genealogy_v2.get_genealogy_2L('APD|ML|UC|QSC', conn) # 
# Map the 'US_id' to '6L_cell_id' using '2L_cell_id' in the 'gen' dataframe
us_to_ps_mapping = dict(zip(gen['2L_cell_id'], gen['6L_cell_id']))
YieldedCells['Cell Status'] = YieldedCells['Cell ID'].map(us_to_ps_mapping)
YieldedCells['Cell Status'] = YieldedCells['Cell Status'].fillna('Waiting')

### YieldedCells has a empty table for tiering metrics, stitch new code HERE!!!!!!!!!!!!
YieldedCells.to_clipboard(index=False)

In [6]:
##Query Metrics for Tiering

import pandas as pd
import numpy as np
from qsdc.client import Client
import met_client as app
import os
import unit_cell_metro_metrics_ZI
import unit_cell_electrical_yield_and_metrics_v2 as uceym
import warnings
from datetime import datetime
import rel_sim as rs
import query_tray_samples_V4 as query_tray_samples
from importlib import reload
from met_client import SearchQuery, ImageAgent
from met_client.constants import AnalysisType, ImageSize
from image_client.client import ImageClient
from image_client.manual_review import convert_manual_reviews_to_dataframe


df_pairing = YieldedCells

###Pull metrics for each yielded cell

## Query HIFI CELL THICKNESS METRICS ##
print("Aquiring Thickness Data")
agent = app.ImageAgent()        
# get US thickness data
df_us_thickness = unit_cell_metro_metrics_ZI.get_thickness_metrics(
        df_pairing["Cell ID"].str.slice(stop=13).unique(), agent
    )
# append "_US" to column names - 
df_us_thickness.columns = [f"{col}_US" for col in df_us_thickness.columns]
df_us_thickness = df_us_thickness.rename(columns={"sample_US": "Cell ID"})
# select desired columns for analysis - updated to use median metrics on the 11-08-24 
#selected_columns_df_us_thickness = df_us_thickness[['Cell ID','10mm_eroded_rect_inside_mean_US','0.5mm_eroded_rect_east_mean_US','0.5mm_eroded_rect_west_mean_US','0.5mm_eroded_rect_north_mean_US','0.5mm_eroded_rect_south_mean_US','center_normalized_0.5mm_eroded_rect_outside_mean_US']]
selected_columns_df_us_thickness = df_us_thickness[['Cell ID','10mm_eroded_rect_inside_median_US','0.5mm_eroded_rect_east_median_US','0.5mm_eroded_rect_west_median_US','0.5mm_eroded_rect_north_median_US','0.5mm_eroded_rect_south_median_US','center_normalized_0.5mm_eroded_rect_outside_median_US']]
# MERGE US DATA
df_processing = df_pairing.merge(selected_columns_df_us_thickness, how="left", on="Cell ID")
# Define the conditions
conditions = [
        df_processing['center_normalized_0.5mm_eroded_rect_outside_median_US'] < 1.1
        ]
    # Define the corresponding values
values = [1]
# Assign values to the 'Thickness' column
df_pairing['Thickness'] = np.where(
    df_processing['center_normalized_0.5mm_eroded_rect_outside_median_US'].notna() & conditions[0], values[0], 3
    )
df_pairing = df_pairing.merge(selected_columns_df_us_thickness[['Cell ID', 'center_normalized_0.5mm_eroded_rect_outside_median_US']], how="left", on="Cell ID")
#print(selected_columns_df_us_thickness)

## Query Edge Wetting Metrics ##
print("Aquiring Edge Wetting Data")
df_edge_wetting_metrics = unit_cell_metro_metrics_ZI.get_edge_wetting_metrics(df_pairing['Cell ID'].str.slice(stop=13).unique(), agent)
df_pairing = df_pairing.merge(df_edge_wetting_metrics[['sample','median_contour_catholyte_pct']], how="left", left_on="Cell ID", right_on = 'sample').drop(columns = 'sample')
#df_pairing['Edge Wetting'] = np.where(df_pairing['median_contour_catholyte_pct'] < 80, 3, 1)


## Query Anode tier for pairing 
print("Aquiring Anode Data")
df_anode_metrics = unit_cell_metro_metrics_ZI.get_anode_tier_A1(df_pairing['Cell ID'].str.slice(stop=13).unique(), agent)
df_pairing['Anode'] = df_pairing['Cell ID'].str.slice(start=0, stop=16).map(df_anode_metrics.set_index('sample')['A1_anode_tier'])


## Query Cathode Misalignment Automated Review ##
with ImageClient(host="image-api.qscape.app") as image_client:
#image_client = ImageClient()  # Use ImageClient as a context manager
    print("Aquiring Cathode Misalignment Data")
    for index, row in df_pairing.iterrows():
        sample_name = row['Cell ID']
        image_search = SearchQuery(
            sample_prefix=sample_name,    
            a_type=AnalysisType.CONTRAST,      
            lregex="nordson_matrix-us-stitched-corners$",   
            )
        image_agent = ImageAgent()
        image_results = image_agent.search(query=image_search)
        # Check if 'cathode_alignment_custom_model_prediction' exists in the image results
        if 'cathode_alignment_custom_model_prediction' in image_results:
            cathodemisalignment = image_results['cathode_alignment_custom_model_prediction']
            if cathodemisalignment.iloc[0] == "go":
                df_pairing.at[index, 'Alignment'] = 1
                #print(f"{sample_name} is a 1 based on CV model")
            elif cathodemisalignment.iloc[0] == "no-go":
                df_pairing.at[index, 'Alignment'] = 3
                #print(f"{sample_name} is a 3 based on CV model")

        #Check manual review of cathode misalignment and edge wetting
        manual_reviews = image_client.get_manual_reviews(samples=[sample_name], include_history=True)
        manualreviewCM = convert_manual_reviews_to_dataframe(manual_reviews, include_modified_date=True)
        if 'cathode_alignment' in manualreviewCM:
            if not manualreviewCM.empty and manualreviewCM['cathode_alignment'].notnull().any():
                df_pairing.at[index, 'Alignment'] = manualreviewCM['cathode_alignment'].iloc[0]
                #print(f"Manual Review corrected {sample_name} cathode misalignment to {manualreviewCM['cathode_alignment'].iloc[0]}")
        if 'edge_wetting' in manualreviewCM:
            if not manualreviewCM.empty and manualreviewCM['edge_wetting'].notnull().any():
                df_pairing.at[index, 'median_contour_catholyte_pct'] = manualreviewCM['edge_wetting'].iloc[0]
                #print(f"Manual Review corrected {sample_name} edge wetting to {manualreviewCM['edge_wetting'].iloc[0]}")

conditions = [
    df_pairing['median_contour_catholyte_pct'] < 80,
    (df_pairing['median_contour_catholyte_pct'] >= 80) & (df_pairing['median_contour_catholyte_pct'] <= 98),
    df_pairing['median_contour_catholyte_pct'] > 98
]
choices = [3, 2, 1]
df_pairing['Edge Wetting'] = np.select(conditions, choices)

# Remove duplicate rows across all columns
#df_pairing = df_pairing.drop_duplicates()


# Select columns that we are considering for tiering
columns_to_consider = [
    'Alignment',
    'Anode',
    'Thickness', 
    'Edge Wetting'
]
# Calculate the minimum value for each row across the selected columns
#df_pairing['Total'] = df_pairing[columns_to_consider].max(axis=1)
# Ensure these columns exist in df_pairing before calculating 'Total'
missing_columns = [col for col in columns_to_consider if col not in df_pairing.columns]
if missing_columns:
    print(f"Warning: The following columns are missing from df_pairing: {missing_columns}")

# Calculate the maximum value for each row across the specified columns
# Use only the columns that exist in df_pairing
valid_columns = [col for col in columns_to_consider if col in df_pairing.columns]
df_pairing['Pairing Group'] = df_pairing[valid_columns].max(axis=1)

## Query electrical metrics for pairing ##
df_electrical_yield_metrics = uceym.get_electrical_yield_and_metrics(df_pairing["Cell ID"])
df_pairing = pd.merge(df_pairing, df_electrical_yield_metrics, left_on='Cell ID', right_on='US_id', how='left')
# df_pairing['Capacity_Difference'] = abs(df_pairing['AMSDischargeCapacity_Co3'] - (-0.4805 * df_pairing['MedDischargeASR_1C'] + 207.0932))

## Query tray location ##
sample_names_group = df_pairing["Cell ID"]
df_tray = query_tray_samples.get_sample_tray(sample_names_group)
df_pairing = pd.merge(df_pairing, df_tray[['sample_name', 'tray_id', 'row_index', 'col_index', 'modified']], left_on='Cell ID', right_on='sample_name', how='left')


# Select and reorder the specified columns
final_columns = [
            "Cell ID", "Batch", "Cell Status", "Pairing Group", 
            "Edge Wetting","median_contour_catholyte_pct", "Thickness", "center_normalized_0.5mm_eroded_rect_outside_median_US", "Alignment", 
            "Anode", "MedDischargeASR_1C", 
            "DischargeCapacity_Co3", "dVdt_delta_fastcharge", 
            "total_rank_score", "tray_id", "row_index", "col_index"
        ]

# Filter the DataFrame to only include the specified columns
rank = df_pairing[final_columns]

# Save the ranked cells to an Excel file
current_date = datetime.now().strftime('%Y-%m-%d')
output_name = f'{current_date}_{batches}_Tier.xlsx'
rank.to_excel(output_name, index=False)
print(f"Saved ranked cells to {output_name}")

Aquiring Thickness Data


100%|██████████| 1/1 [00:01<00:00,  1.48s/it]


Aquiring Edge Wetting Data


100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


Aquiring Anode Data


100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


Aquiring Cathode Misalignment Data


Saved ranked cells to 2024-12-02_UCB004A_Tier.xlsx


In [7]:
## Format and prepare dataframes for TOPSIS Rank and Grouping

#Rename 
TieredCells = rank

#Keep Tier 1 Cells that have not been grouped
TieredCells = TieredCells[TieredCells["Cell Status"] == "Waiting"]

#Keep Tier 1 Cells to commence pairing of Tier 1 groups
TieredOneCells = TieredCells[TieredCells['Pairing Group'] == 1].copy()
TieredTwoThreeCells = TieredCells[TieredCells['Pairing Group'] != 1].copy()

# Remove duplicate rows across all columns
TieredOneCells = TieredOneCells.drop_duplicates()
TieredTwoThreeCells = TieredTwoThreeCells.drop_duplicates()

In [8]:
##Calculate Cell Score and Ranking for Tier 1 Cells using TOPSIS method
# Define the criteria and their corresponding optimization directions
criteria = ['median_contour_catholyte_pct', 'center_normalized_0.5mm_eroded_rect_outside_median_US', 'MedDischargeASR_1C', 'DischargeCapacity_Co3'] 
weights = [0.15, 0.25, 0.35, 0.25]  # Equal weights (adjust if needed)
optimization_directions = ['High', 'Proximity', 'Low', 'High']  # 'Proximity' for 'Thickness'
target_criteria = ['center_normalized_0.5mm_eroded_rect_outside_median_US']  # Criteria optimized for proximity
targets = [1.0]  # Target value for 'Thickness'


def topsis_with_target_Tier1(df, criteria, weights, optimization_directions, target_criteria=None, targets=None):
    
    # Ensure numeric data for criteria
    df[criteria] = df[criteria].apply(pd.to_numeric, errors='coerce')

    normalized_matrix = df[criteria].copy()
    
    # Step 1: Normalize the Decision Matrix
    for col in criteria:
        if target_criteria and col in target_criteria:
            # Calculate absolute deviation from target
            target = targets[target_criteria.index(col)]
            normalized_matrix[col] = np.abs(df[col] - target)  # Deviation
        else:
            # Standard normalization
            normalized_matrix[col] = normalized_matrix[col] / np.sqrt((normalized_matrix[col]**2).sum())
    
    # Step 2: Weight the Criteria
    weighted_matrix = normalized_matrix * weights
    
    # Step 3: Identify Ideal and Negative Ideal Solutions
    ideal_solution = []
    negative_ideal_solution = []
    for col, direction in zip(criteria, optimization_directions):
        if target_criteria and col in target_criteria:
            # For deviation-based criteria, minimize the deviation
            ideal_solution.append(weighted_matrix[col].min())
            negative_ideal_solution.append(weighted_matrix[col].max())
        else:
            if direction == 'High':
                ideal_solution.append(weighted_matrix[col].max())
                negative_ideal_solution.append(weighted_matrix[col].min())
            elif direction == 'Low':
                ideal_solution.append(weighted_matrix[col].min())
                negative_ideal_solution.append(weighted_matrix[col].max())
    
    ideal_solution = np.array(ideal_solution)
    ideal_solution = np.array([
        weighted_matrix.iloc[:, idx].max() if direction == "maximize" else weighted_matrix.iloc[:, idx].min()
        for idx, direction in enumerate(optimization_directions)
    ])

    negative_ideal_solution = np.array(negative_ideal_solution)
    negative_ideal_solution = np.array([
        weighted_matrix.iloc[:, idx].min() if direction == "maximize" else weighted_matrix.iloc[:, idx].max()
        for idx, direction in enumerate(optimization_directions)
    ])
    
    # Step 4: Calculate Distances
    distances_to_ideal = np.sqrt(((weighted_matrix - ideal_solution)**2).sum(axis=1))
    distances_to_negative = np.sqrt(((weighted_matrix - negative_ideal_solution)**2).sum(axis=1))
    
    # Step 5: Calculate TOPSIS Score
    topsis_score = distances_to_negative / (distances_to_ideal + distances_to_negative)
    return topsis_score

# Calculate TOPSIS scores
TieredOneCells['total_rank_score'] = topsis_with_target_Tier1(TieredOneCells, criteria, weights, optimization_directions, target_criteria, targets)
# Rank alternatives by TOPSIS score (higher score is better)
TieredOneCells['Rank'] = TieredOneCells['total_rank_score'].rank(ascending=False)
# Arrange Cells by best ranking at the top
TieredOneCells.sort_values(by='Rank', ascending=True, inplace=True)


In [9]:
df = TieredOneCells
df = df.reset_index(drop=True)
df['Rank'] = (df.index + 1).astype(int)  # Add 1-based rank as integers
remainder = df.copy()  # Start with a copy of the input DataFrame
retries = 0  # Track retries for reprocessing the remainder

n = 3

#Initialize variable and dataframe 
groupnumber = 1  # Initialize group number
df_final = pd.DataFrame()  # Initialize an empty DataFrame for the final result
grouped_list = [] # List to collect all the groups for final concatenation
cellstart = 0

# Keep track of processed indices
processed_indices = set()

while len(remainder) >= n + cellstart:
    # Find the next candidate group by slicing remainder
    candidate_group = remainder.iloc[cellstart:cellstart+n].copy()
    print(f"Processing group starting at rank {candidate_group['Rank'].tolist()}: {candidate_group['Cell ID'].tolist()}")

    # Calculate criteria for the group
    asr_range = candidate_group['MedDischargeASR_1C'].max() - candidate_group['MedDischargeASR_1C'].min()
    asr_mean_min = candidate_group['MedDischargeASR_1C'].mean() - candidate_group['MedDischargeASR_1C'].min()

    print(f"ASR range: {asr_range}, ASR mean-min: {asr_mean_min}")

    # Check if group satisfies criteria and replace cells if necessary
    if asr_range >= 2.0:
        # Replace the highest ASR cell
        max_idx = candidate_group['MedDischargeASR_1C'].idxmax()
        problem_cell_id = candidate_group.loc[max_idx, 'Cell ID']
        print(f"ASR range is too high at {asr_range}, replacing problem cell: {problem_cell_id}")
        # Replace the highest ASR cell here (you can replace it based on your logic)
        i_range = 0

        while asr_range >=2.0:
            if n + i_range >= len(remainder):  # Check if the index is valid
                print(f"No more rows in remainder to replace {problem_cell_id}.")
                cellstart = cellstart +1
                break
            candidate_group.loc[max_idx] = remainder.iloc[n + i_range] # Replace the highest ASR cell
            replacement_cell_id = remainder.iloc[n + i_range]['Cell ID']  # Cell ID of the replacement cell
            print(f"Replacing {problem_cell_id} with {replacement_cell_id}")

            # Check for duplicates in 'Cell ID' within the candidate group
            duplicate_count = candidate_group['Cell ID'].duplicated(keep=False).sum()
            while duplicate_count > 0:
                print(f"Found {duplicate_count} duplicate(s) of 'Cell ID' in candidate_group. Incrementing i_range.")
                i_range += 1  # Increment if there are duplicates
                if n + i_range < len(remainder):
                    candidate_group.loc[max_idx] = remainder.iloc[n + i_range]
                    replacement_cell_id = remainder.iloc[n + i_range]['Cell ID']  # Replacement cell ID
                    print(f"Replacing {problem_cell_id} with {replacement_cell_id}")
                    duplicate_count = candidate_group['Cell ID'].duplicated(keep=False).sum()

            asr_range = candidate_group['MedDischargeASR_1C'].max() - candidate_group['MedDischargeASR_1C'].min()
            asr_mean_min = candidate_group['MedDischargeASR_1C'].mean() - candidate_group['MedDischargeASR_1C'].min()
            print(f"ASR range: {asr_range}, ASR mean-min: {asr_mean_min}")
            i_range = i_range+1

    if asr_mean_min >= 0.5:
        # Replace the lowest ASR cell
        min_idx = candidate_group['MedDischargeASR_1C'].idxmin()
        problem_cell_id = candidate_group.loc[min_idx, 'Cell ID']
        print(f"Minimum ASR is too low at difference of {asr_mean_min}, replacing problem cell: {problem_cell_id}")
        # Replace the lowest ASR cell here (you can replace it based on your logic)
        i_mean_min = 0

        while asr_mean_min >= 0.5:
            if n + i_mean_min >= len(remainder):  # Check if the index is valid
                print(f"No more rows in remainder to replace {problem_cell_id}.")
                cellstart = cellstart +1
                break
            candidate_group.loc[min_idx] = remainder.iloc[n + i_mean_min]
            replacement_cell_id = remainder.iloc[n + i_mean_min]['Cell ID']  # Replacement cell ID
            print(f"Replacing {problem_cell_id} with {replacement_cell_id}")

            # Check for duplicates in 'Cell ID' within the candidate group
            duplicate_count = candidate_group['Cell ID'].duplicated(keep=False).sum()
            while duplicate_count > 0:
                print(f"Found {duplicate_count} duplicate(s) of 'Cell ID' in candidate_group. Incrementing i_mean_min.")
                i_mean_min += 1  # Increment if there are duplicates
                if n + i_mean_min < len(remainder):
                    candidate_group.loc[min_idx] = remainder.iloc[n + i_mean_min]
                    replacement_cell_id = remainder.iloc[n + i_mean_min]['Cell ID']  # Replacement cell ID
                    print(f"Replacing {problem_cell_id} with {replacement_cell_id}")
                    duplicate_count = candidate_group['Cell ID'].duplicated(keep=False).sum()
            
            asr_range = candidate_group['MedDischargeASR_1C'].max() - candidate_group['MedDischargeASR_1C'].min()
            asr_mean_min = candidate_group['MedDischargeASR_1C'].mean() - candidate_group['MedDischargeASR_1C'].min()
            print(f"ASR range: {asr_range}, ASR mean-min: {asr_mean_min}")
            i_mean_min = i_mean_min+1


    if asr_range < 2.0 and asr_mean_min < 0.5:
        print(f"Established group starting at rank {candidate_group['Rank'].tolist()}: {candidate_group['Cell ID'].tolist()}")
        # Assign the group number
        candidate_group['Group Number'] = groupnumber
        # Add the rows to the final list
        grouped_list.append(candidate_group)
        # Drop rows in "remainder" where "Cell ID" matches any in "candidate_group"
        remainder = remainder[~remainder['Cell ID'].isin(candidate_group['Cell ID'])]
        # Update processed indices
        processed_indices.update(candidate_group.index)
        # Update group number for the next group
        groupnumber += 1
        # Print status
        print(f"Rows in remainder after processing: {len(remainder)}")


# Check if grouped_list contains valid DataFrames
if not grouped_list:
    print("No valid groups were formed. Creating an empty DataFrame.")
    df_final = pd.DataFrame()  # Handle case where no groups are created
else:
    df_final = pd.concat(grouped_list, ignore_index=True)
# Add 'Remainder' label to remaining rows, if applicable
if not remainder.empty:
    remainder['Group Number'] = 'Remainder'
    df_final = pd.concat([df_final, remainder], ignore_index=True)

# Add 'Remainder' label to remaining rows
#remainder['Group Number'] = 'Remainder'

# Add the remainder rows to df_final
#df_final = pd.concat([df_final, remainder], ignore_index=True)

# Format final DataFrame
df_final = df_final.reset_index(drop=True)
df_final = df_final.drop(columns=['median_contour_catholyte_pct', 'center_normalized_0.5mm_eroded_rect_outside_median_US','Rank']) #drop rows that we don't need to show in the spreadsheet
df_final = df_final[['Group Number'] + [col for col in df_final.columns if col != 'Group Number']] #move 'Group Number' to first column

# Print the final DataFrame
print(f"Final DataFrame has {df_final.shape[0]} rows.")

# Save the ranked cells to an Excel file
current_date = datetime.now().strftime('%Y-%m-%d')
output_name = f'{current_date}_{batches}_Tier_1_Grouped.xlsx'
df_final.to_excel(output_name, index=False)

Processing group starting at rank [1, 2, 3]: ['UCB004AA-US00-08', 'UCB004AA-US00-04', 'UCB004AA-US00-22']
ASR range: 1.157399999999999, ASR mean-min: 0.4656333333333329
Established group starting at rank [1, 2, 3]: ['UCB004AA-US00-08', 'UCB004AA-US00-04', 'UCB004AA-US00-22']
Rows in remainder after processing: 0
Final DataFrame has 3 rows.


In [10]:
# Function to rearrange rows in pyramid order within each group
def pyramid_order(group):
    # Sort values within the group
    sorted_group = group.sort_values(by='MedDischargeASR_1C', ascending=True)
    # Rearrange in pyramid order
    pyramid = []
    while len(sorted_group) > 0:
        if len(pyramid) % 2 == 0:  # Add largest remaining value to the middle
            pyramid.append(sorted_group.iloc[-1])
            sorted_group = sorted_group[:-1]
        else:  # Add smallest remaining value to the sides
            pyramid.insert(0, sorted_group.iloc[0])
            sorted_group = sorted_group[1:]
    # Reconstruct DataFrame
    return pd.DataFrame(pyramid)

# Apply the function to each group
df_pyramid = df_final.groupby('Group Number', group_keys=False).apply(pyramid_order)

# Reset the index if needed
df_pyramid = df_pyramid.reset_index(drop=True)

# Save the ranked cells to an Excel file
current_date = datetime.now().strftime('%Y-%m-%d')
output_name = f'{current_date}_{batches}_Tier_1_Grouped_Pyramid_Order.xlsx'
df_pyramid.to_excel(output_name, index=False)


: 